
# Section 8, Lecture 2: Hands-on LangGraph: Build a Reliable AI Workflow for Business

Welcome to Lecture 2 of Section 8! In the last lecture, we introduced LangGraph and saw how it can help you build reliable AI workflows. Now, we’re getting hands-on. In this notebook, you’ll create your own LangGraph workflow to automate a real-world business task: triaging customer support inquiries. By the end, you’ll have a working system that ensures reliability, saves time, and keeps your customers happy.

### What You’ll Build
- A LangGraph workflow with three nodes:
  - **Classify Node**: Determines the inquiry type (e.g., question, complaint, urgent).
  - **Route Node**: Decides where to send the inquiry (AI agent or human).
  - **Respond Node**: Sends an AI response or flags for human review.
- The workflow will include error handling (e.g., retries) to ensure reliability.

### Why This Matters for Business
Triaging customer support inquiries manually takes hours and risks delays. With LangGraph, you’ll automate this process with reliability, ensuring faster responses, fewer errors, and a better customer experience.

Let’s get started!


## ⚙️ Step 1: Installing LangGraph

Before we build anything, we need to make sure LangGraph is installed and ready to go.

### 📦 What We're Doing:
- Verifying whether the LangGraph package is installed in your environment.
- If not installed, you’ll need to install it manually using:
  ```bash
  uv pip install langgraph
  ```

### 💼 Why This Matters:
LangGraph is the backbone of your workflow automation—just like a project management system ensures every step gets done reliably.

### 🛠️ Pro Tip:
If you're not seeing a version output, double-check that your virtual environment is active and that you ran the install command in the terminal.


In [5]:
# If you haven’t installed LangGraph yet, run this in your terminal (not here):
# uv pip install langgraph
# Make sure your virtual environment is activated:
# Mac/Linux: source .venv/bin/activate
# Windows: .venv\Scripts\Activate

# Verify the installation
try:
    import langgraph
    from importlib.metadata import version
    print("✅ LangGraph is installed! Version:", version("langgraph"))
except ImportError:
    print("❌ LangGraph not found. Please install it using the command above.")
except Exception as e:
    print("⚠️ An error occurred while checking version:", e)


✅ LangGraph is installed! Version: 0.3.30


## 🔐 Step 2: Load Your OpenAI API Key

To run intelligent workflows, we need GPT-4 to power decision-making. We’ll access it securely using your OpenAI API key.

### 🧩 What We're Doing:
- Loading the `.env` file that stores your API credentials.
- Verifying that the key is accessible within the notebook.

### 💼 Why This Matters:
Securely storing and loading your API keys is a **best practice in real-world production systems**, where leaking keys can have cost and security implications.

### 🛠️ Pro Tip:
Your `.env` file should contain a line like this:
`OPENAI_API_KEY=your-key-here`
Never hardcode it directly into your script or notebook.

In [6]:

from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verify the key
if openai_api_key:
    print("API key loaded successfully!")
else:
    print("API key not found. Make sure your .env file has OPENAI_API_KEY set.")


API key loaded successfully!


## 🧠 Step 3: Setting Up the LangGraph Workflow

Let’s lay the foundation for your AI-powered customer support triage system by creating a LangGraph workflow.

### 🧩 What We're Doing:
- Creating a `StateGraph`, which lets us define and manage each step in the workflow.
- Initializing the GPT-4 model as our language reasoning engine.
- Creating a `WorkflowState` schema to store state across steps.

### 💼 Why This Matters:
In real business systems, each process must reliably move from one stage to another. LangGraph’s StateGraph ensures your flow is consistent and recoverable.

### 📊 Real-World Use Case:
Think of this like an automated ticket system—once set up, it guarantees tickets are never lost or stuck.


In [7]:

from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from typing import Dict, Any

# Initialize the LLM (GPT-4)
llm = ChatOpenAI(
    model="gpt-4",
    api_key=openai_api_key
)

# Define the state to track the workflow
class WorkflowState(dict):
    inquiry: str
    inquiry_type: str
    response: str
    needs_human: bool

# Create the workflow graph
workflow = StateGraph(WorkflowState)

print("Workflow set up successfully!")


Workflow set up successfully!


## 🔄 Step 4: Defining Nodes and Workflow Logic

This is where your automation logic comes to life. Each **node** represents a step in the decision-making process.

### 🤖 Node Roles:
- **Classify Node**: Uses GPT-4 to label the inquiry.
- **Route Node**: Decides whether to escalate or respond.
- **Respond Node**: Drafts a reply or flags for human review.

### 🔁 Flow Control:
We connect nodes with **edges**, like arrows in a flowchart, that define how information flows from one step to another.

### 🧩 Why It Matters:
This level of structure is critical in systems where mistakes are costly—like customer complaints or legal queries. LangGraph ensures reliable, recoverable execution.


In [8]:

# Node 1: Classify the inquiry
def classify_inquiry(state: WorkflowState) -> WorkflowState:
    inquiry = state["inquiry"]
    prompt = f"Classify this customer inquiry as a question, complaint, or urgent issue: {inquiry}"
    response = llm.invoke(prompt).content
    state["inquiry_type"] = response.lower()
    return state

# Node 2: Route the inquiry
def route_inquiry(state: WorkflowState) -> WorkflowState:
    inquiry_type = state["inquiry_type"]
    state["needs_human"] = "complaint" in inquiry_type or "urgent" in inquiry_type
    return state

# Node 3: Respond or escalate
def respond_or_escalate(state: WorkflowState) -> WorkflowState:
    if state["needs_human"]:
        state["response"] = "Flagged for human review."
    else:
        prompt = f"Draft a brief response to this customer inquiry: {state['inquiry']}"
        state["response"] = llm.invoke(prompt).content
    return state

# Add nodes to the workflow
workflow.add_node("classify", classify_inquiry)
workflow.add_node("route", route_inquiry)
workflow.add_node("respond", respond_or_escalate)

# Define edges (flow between nodes)
workflow.set_entry_point("classify")
workflow.add_edge("classify", "route")
workflow.add_edge("route", "respond")
workflow.add_edge("respond", END)

# Compile the workflow
app = workflow.compile()

print("Nodes and edges defined successfully!")


Nodes and edges defined successfully!


## 🚀 Step 5: Running the Workflow

Now let’s run your LangGraph pipeline on a **real-world inquiry**: “My order is delayed—what’s going on?”

### 🧪 What Happens:
1. The inquiry is classified (e.g., as a complaint).
2. The system routes it to either an agent or flags it.
3. A response is generated or it's escalated to a human.

### 💼 Why This Matters:
You just built a reliable triage bot—imagine deploying this inside Zendesk or Freshdesk. It ensures **zero tickets fall through the cracks**.

### ✏️ Try This:
Replace the sample inquiry with something like “I want to cancel my subscription” and see how the flow adapts.


In [9]:

initial_state = WorkflowState(
    inquiry="My order is delayed—what’s going on?",
    inquiry_type="",
    response="",
    needs_human=False
)

result = app.invoke(initial_state)

# Print the final output
print("Final Output:\n", result["response"])


Final Output:
 Dear Customer,

We sincerely apologize for the delay in your order. We understand how frustrating this can be. Upon checking, we found that there has been an unexpected delay in our supply chain which has affected the delivery of some orders. We are working tirelessly to resolve this issue and get your order to you as soon as possible. We appreciate your patience and understanding during this time.

Best regards,
[Your Name]


## 🎓 Wrap-Up and What’s Next

Congratulations! You’ve just built your **first reliable AI workflow using LangGraph**.

### 💡 What You Built:
- A structured graph with intelligent steps (nodes).
- A decision-making pipeline with GPT-4.
- A workflow that can **scale, retry, and never miss a beat**.

### 🧠 Reflect:
- How could this apply in your business?
- Could you use a similar flow for onboarding, refunds, or lead scoring?

### 👉 Next:
In the next lecture, we’ll enhance this with **memory and branching logic**, enabling smarter decisions based on past interactions.
